In [1]:
import pickle
import numpy as np
from scipy.sparse import csr_matrix

with open("data/mol_bits.pkl", "rb") as f:
    mol_feats = pickle.load(f)


# Extracting unique features for sparse matrix dims
all_feats = set()
for feats_set in mol_feats.values():
    all_feats.update(feats_set)

n_feats = max(all_feats) + 1  # 0-based indexing

# Creating sparse matrix mol feats
rows, cols = [], []
for i, (uniprot_id, feats_set) in enumerate(mol_feats.items()):
    rows.extend([i] * len(feats_set))
    cols.extend(feats_set)

mol_ids = list(mol_feats.keys())

mol_feats = csr_matrix(
    (np.ones(len(rows)), (rows, cols)), shape=(len(mol_feats), n_feats)
)

mol_feats = mol_feats.T  # Transposing to have features as rows

# LSH implementation
- MinHash
- Buckets


In [2]:
import hashlib


def create_hash(vector_size=mol_feats.shape[0]):
    return np.random.permutation(vector_size)


def get_signature_matrix(sparse_mat, sig_size=50):
    permutations = np.zeros((sparse_mat.shape[0], sig_size), dtype=int)
    for i in range(sig_size):
        permutations[:, i] = create_hash()

    # signature matrix
    signature_matrix = np.zeros((permutations.shape[1], sparse_mat.shape[1]))

    # minhashing
    for i in range(permutations.shape[1]):
        perm = permutations[:, i]
        sigs = sparse_mat[perm, :].argmax(axis=0)
        signature_matrix[i, :] = sigs

    return signature_matrix


def split_sig(sig: np.array, b: int):
    return np.vsplit(sig, b)


def hash_bands_to_buckets(bands, num_buckets=1000):

    hashed_bands = {}

    n_bands = len(bands)

    for i in range(n_bands):
        band = bands[i]
        for col in range(band.shape[1]):
            col_hash = hashlib.sha256(band[:, col].tobytes()).hexdigest()
            bucket = int(col_hash, 16) % num_buckets

            if bucket not in hashed_bands.keys():
                hashed_bands[bucket] = set()
            hashed_bands[bucket].add(col)

    return hashed_bands


def jaccard_similarity(ids, data, mol_ids):
    id1, id2 = ids
    set1 = data[mol_ids[id1]]
    set2 = data[mol_ids[id2]]
    return len(set1.intersection(set2)) / len(set1.union(set2))


def fetch_pairs(buckets, threshold=0.8):
    candidates = []
    for proteins in buckets.values():
        if len(proteins) > 1 and len(proteins) < 3:
            sim = jaccard_similarity(tuple(proteins))
            if sim >= threshold:
                candidates.append(tuple(proteins))
    return candidates

In [3]:
mol_feats

<2048x73865 sparse matrix of type '<class 'numpy.float64'>'
	with 5655190 stored elements in Compressed Sparse Column format>

In [4]:
# SIG
SIG = get_signature_matrix(mol_feats, sig_size=100)

# split signature into bands
b = 10
bands = split_sig(SIG, b)

# hash bands to buckets
buckets = hash_bands_to_buckets(bands, num_buckets=1_000_000)

candidates = fetch_pairs(buckets, threshold=0.8)

In [6]:
# hash bands to buckets
buckets = hash_bands_to_buckets(bands, num_buckets=1_000_000)

In [7]:
buckets

{692071: {0},
 671828: {1, 2, 48902},
 555745: {3, 69099, 71866},
 10627: {4, 15},
 488106: {5, 3215, 38402},
 553865: {6},
 321395: {7},
 249920: {8, 19310},
 474602: {9},
 498538: {10, 5934, 52342},
 194688: {11, 26272},
 433367: {12, 61995, 69769},
 584879: {13, 17607},
 336179: {14},
 721959: {16},
 391399: {17, 26, 27},
 351766: {18},
 865776: {19},
 850254: {20},
 403688: {21, 30187, 40569, 40582, 71970},
 348851: {22, 22367},
 333105: {23},
 843190: {24},
 106201: {25},
 940389: {28, 29, 30},
 636979: {31, 32, 33, 44, 3611, 64390},
 803902: {34},
 133562: {35, 21991},
 814358: {36, 68, 78, 33903},
 907231: {37, 13706, 19984},
 461011: {38, 45572, 45577},
 270331: {39, 45357},
 375281: {40},
 123844: {41},
 618193: {42, 31359, 39202},
 345932: {43, 45},
 959278: {46},
 424069: {47},
 242843: {48, 17850},
 998387: {49},
 461541: {50, 37809, 37810, 37813, 67545},
 582297: {51, 65341},
 571045: {52},
 505297: {53, 21601, 45732},
 371760: {54, 16122, 30831},
 618680: {55},
 700402: {